# [Module 3.4.2] Custom Docker Image 사용하여 학습

이 노트북은 스크립트 모드로 학습을 수행 합니다.
여기서는 다음과 같은 작업을 합니다.

- 학습할 데이타를 S3로 지정
- Train 학습 파리미터 설정
- Estimator를 생성하고 Train Docker Image 경로 설정
- train_instance_type 을 'ml.p3.2xlarge' 인스턴스로 설정
- 학습
- S3에 생성된 모델 아티펙트 확인


In [88]:
%store -r

In [89]:
import os
import sagemaker
import boto3
from sagemaker.tensorflow import TensorFlow

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

## 입력 데이터 설정

In [90]:
s3_input_train_data = sagemaker.s3_input(s3_data=processed_train_data_s3_uri, 
                                         distribution='ShardedByS3Key') 
s3_input_validation_data = sagemaker.s3_input(s3_data=processed_validation_data_s3_uri, 
                                              distribution='ShardedByS3Key')
s3_input_test_data = sagemaker.s3_input(s3_data=processed_test_data_s3_uri, 
                                        distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-18-03-14-58-057/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-18-03-14-58-057/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-18-03-14-58-057/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


In [91]:
import uuid

checkpoint_s3_prefix = 'checkpoints/{}'.format(str(uuid.uuid4()))
checkpoint_s3_uri = 's3://{}/{}/'.format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

s3://sagemaker-ap-northeast-2-343441690612/checkpoints/30d6ed2e-ff50-4b67-9ec7-6c16efced81e/


In [92]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

## Parameters

In [93]:
epochs= 400

learning_rate= 4e-4
epsilon=0.00000001

steps = 100

train_steps_per_epoch= steps
validation_steps= int(steps / 2)
test_steps= int(steps / 2)

train_batch_size=128
validation_batch_size=128
test_batch_size=128



train_instance_count=2 # modified by gonsoo
train_instance_type='ml.p3.2xlarge'
train_volume_size=1024

use_xla=True
use_amp=True

max_seq_length = 32
freeze_bert_layer= True

enable_checkpointing=True
input_mode='Pipe'
run_validation=True
run_test=True

아래와 같이 Train 용의 Container를 ECR 콘솔에서 복사하여 사용 함

![ECR-Training-Container](img/ecr-train-container.png)

**Custom Docker Image를 생성하였다면, 위의 ECR 콘솔에 가서 이미지 경로를 복사해서 아래를 대체 하세요.**
아래 셀을 바로 실행하면 에러 발생 합니다.

In [94]:
ecr_image = "343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/bert2tweet:latest"

In [95]:
from sagemaker.estimator import Estimator


estimator = Estimator( image_name = ecr_image,
                       role=sagemaker.get_execution_role(),
                       train_instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       train_instance_type=train_instance_type,
                       train_volume_size=train_volume_size,
                       checkpoint_s3_uri=checkpoint_s3_uri, # Not support in local mode
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'run_validation':run_validation,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_checkpointing': enable_checkpointing
                                        },
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions
                      )

In [96]:
inputs={'train': s3_input_train_data, 
        'validation': s3_input_validation_data,
         'test': s3_input_test_data
              }

estimator.fit(inputs,wait=False)         

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p3.8xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 2 Instances. Please contact AWS support to request an increase for this limit.

## Train Job Status 확인

In [ ]:
latest_training_job = estimator.latest_training_job

In [ ]:
latest_training_job.describe()

In [ ]:
estimator.latest_training_job.wait(logs=False)

## 생성된 모델 아티펙트 확인

In [ ]:
training_job_name = estimator.latest_training_job.job_name
model_artifact_path = "s3://{}/{}/{}".format(bucket,training_job_name,'output' )
! aws s3 ls {model_artifact_path} --recursive

In [ ]:
%store training_job_name